<a href="https://colab.research.google.com/github/reema8mes/reema8mes/blob/main/transfer_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.chdir('/content/drive/MyDrive/fdp')

In [ ]:
import tensorflow as tf
from tensorflow import keras
tf.random.set_seed(42)
import numpy as np
np.random.seed(42)
import matplotlib.pyplot as plt
%matplotlib inline
#for acessing input file use glob
import glob
import PIL
from PIL import Image

In [ ]:
! unzip '/content/drive/MyDrive/fdp/Flower_dataset.zip' -d '/content/drive/MyDrive/fdp/output'

In [ ]:
imgFiles=glob.glob('/content/drive/MyDrive/fdp/output/tiny_FR/*/*.jpg')
for items in imgFiles[:8]:
  print(items)


In [ ]:
print(len(imgFiles))

In [ ]:
X = []
y = []

for fName in imgFiles:
  
  # Prepare the dataset and populate X and y
  X_i = Image.open(fName) # tiny_FR/sunflower/1715303025_e7065327e2.jpg (500, 333)
  X_i = X_i.resize((299,299)) # To make them approriate to Xception model when using Transfer Learning 
  X_i = np.array(X_i) / 255.0 # Normalize to range 0.0 to 1.0 
  X.append(X_i)

  label = fName.split("/") # ['tiny_FR', 'sunflower', '1715303025_e7065327e2.jpg'], list containing substrings
  y_i = label[-2] # 'sunflower'
 
  y.append(y_i)

#processed all  image files and corresponding intensity values and labels stored in X and y respectively 

In [ ]:
print(set(y)) # python function that lists unique values in the list. # these are string labels

# but the network can only deal with numeric data.  

In [ ]:
print(set(Y))

In [ ]:
from sklearn.preprocessing import LabelEncoder
lEncoder = LabelEncoder()
y = lEncoder.fit_transform(y)
print(set(y))

print(lEncoder.classes_) #classes_ is a member variable in the class


In [ ]:
X = np.array(X)
y = np.array(y)

print(X.shape)
print(y.shape)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    stratify=y, random_state=42)

print("X_train_shape: {}".format(X_train.shape))
print("X_test_shape: {}".format(X_test.shape))

In [ ]:
#standard scaling 
mu = X_train.mean()
std = X_train.std()

X_train_std = (X_train-mu)/std
X_test_std = (X_test-mu)/std

In [ ]:
# Retrieve the Xception model from Keras
#view the entire model  
Xception_model = keras.applications.Xception()

tf.keras.utils.plot_model(
    Xception_model,
    to_file='Xmodel.png',
    show_shapes=False,
    show_layer_names=False,
    rankdir='LR',
    expand_nested=False,
    dpi=96
)

In [ ]:
#https://keras.io/api/applications/

In [ ]:
# Take only the feature extractor part

base_model = keras.applications.Xception(weights='imagenet', 
                                                  include_top=False)

# Visualize the extractor part for transfer learning

tf.keras.utils.plot_model(
    base_model,
    to_file='baseModel.png',
    show_shapes=False,
    show_layer_names=False,
    rankdir='LR',
    expand_nested=False,
    dpi=96
)

In [ ]:
# Make the weights/parameters of fetaure extractor non-trainable
# Freeze the weights - will not be updated during backpropagation
for layer in base_model.layers:
  layer.trainabe = False


In [ ]:
#Add classifier part
#Global Average Pooling - 2D
#Output layer (5 units (because 5-class classificaiton problem)
# Adding classifier
#Inspect how flattening is made in the pretrained model. In Xception, they use GAP
global_pool = keras.layers.GlobalAveragePooling2D()(base_model.output)

output_ = keras.layers.Dense(units=5, activation='softmax')(global_pool)

model_TL = keras.models.Model(inputs=[base_model.input], outputs=[output_])

# Visualize the complete model
tf.keras.utils.plot_model(
    model_TL,
    to_file='Model_TL.png',
    show_shapes=False,
    show_layer_names=False,
    rankdir='LR',
    expand_nested=False,
    dpi=96
)
     

In [ ]:
model_TL.compile(loss='sparse_categorical_crossentropy', 
                 optimizer='adam', 
                 metrics=['accuracy'])
#introduce callbacks to save the best model


callbacks_TL = [            
             keras.callbacks.ModelCheckpoint("bestTL.h5",
                                             monitor='val_accuracy',verbose=1,
                                             save_weights_only=True,
                                             save_best_only=True)]


history_TL = model_TL.fit(x = X_train_std, y = y_train, epochs=50, 
                          validation_split=0.1, batch_size=16, callbacks=callbacks_TL)

In [ ]:
keys = ['accuracy', 'val_accuracy']
progress = {k:v for k,v in history_TL.history.items() if k in keys}

import pandas as pd
pd.DataFrame(progress).plot()

plt.xlabel("epochs")
plt.ylabel("accuracy")

plt.grid(True)
plt.show()
     